# **Forecasting the Impact of Climate Change on Great Britain’s Offshore Wind Generation Using Deep Learning**

### Group 3:
**Candidate Codes:** (KXBS7, PNXF3, NQMJ9, NCJB4, MMVQ8, KVQS4)

## **Table of Contents**

1. Introduction

2. Data Sources

3. Data Extraction

4. Data Preprocessing

5. Baseline Modelling

6. Model Development and Evaluation

7. Hyperparameter Tuning

8. Model application

## **Introduction**
Climate change is exerting a significant influence on global weather patterns, as evidenced by the increasing frequency of heatwaves, wildfires, and other extreme weather events. As renewable energy has become central to decarbonization strategies within the power sector, it is critical to understand the long-term impacts of climate change on the performance of renewable energy assets, particularly given that changes in climate patterns can directly affect their energy output.

The trajectory of future climate conditions depends largely on present-day energy policies and the rate of technological advancement. Several climate models have been developed to project future scenarios based on varying assumptions. Among these, the Representative Concentration Pathways (RCPs), developed by the climate science community, provide standardized scenarios that relate different policy choices to levels of radiative forcing.

The aim of this study is to evaluate the impacts of climate change on offshore wind generation in Great Britain under different RCP scenarios. This study focuses on offshore wind in particular as it constitutes a major component of the renewable energy mix in Great Britain. Offshore wind farms have experienced rapid development due to the region’s favourable wind conditions and the advantages offshore sites offer in terms of consistent wind speeds.

A deep learning approach is applied in conducting this study under two specific scenarios:

1. **Representative Concentration Pathway (RCP) 8.5 — a high-emission, business-as-usual scenario (equivalent to shared socio-economic pathway SSP5-8.5)**
Often called "middle of the road" pathway, this scenario is characterized by strong climate policies, reduction in carbon-intensity of fuels and a sharp decline in greenhouse gas (GHG) emissions leading to radiative forcing of 2.6W/m2 by 2100 (hence the name ssp2.6) or 1.5-2 degrees of warming by 2100, consistent with the Paris accord.

2. **Representative Concentration Pathway (RCP) 2.6 — a low-emission, best-case scenario (equivalent to shared socio-economic pathway SSP1-2.6).**
Often called "fossil fuel development" (Business as usual) pathway, this scenario is characterized by little to no climate mitigation efforts, increased use of carbon-intensive fossil fuels driven by rapid econonc growth and energy demand, and continually increasing greenhouse gas (GHG) emissions leading to radiative forcing of 8.5W/m2 by 2100 (hence the name ssp8.5) or 4 degrees or more of warming by 2100, which violates the Paris accord agreements.

The findings will enable policymakers and stakeholders to better evaluate the resilience of the United Kingdom’s offshore wind strategy and to identify necessary contingencies to maintain long-term energy security.

___


### **Project Aim**
To investigate the impact of climate change on Great Britain's offshore wind generation by developing a deep learning-based forecasting model under the RCP 2.6 and 8.5 scenarios. This analysis is conducted at the wind farm level to extract granular and actionable insights.

### **Objectives**
1. **Data Collection and Preprocessing:**  
Gather and preprocess granular wind farm-level data on offshore wind generation, historical meteorological variables, and future climate projections.

2. **Exploratory Data Analysis:**  
Analyze wind farm-specific data to identify key factors (local wind speeds, seasonal trends, and climate variations) influencing offshore wind generation.

3. **Model Development:**  
Develop a deep learning model capable of forecasting wind farm-level generation based on historical wind generation and climate data.

4. **Scenario-Based Forecasting:**  
Forecast future wind generation at individual wind farm sites under RCP 8.5 and RCP 2.6 climate scenarios using climate data projections.

5. **Impact Analysis:**  
Analyze the impact of climate change on individual wind farms, examining how different climate scenarios affect regional performance and the overall capacity of the offshore wind sector in Great Britain.

## **Methodology**
The approach this study takes is to train a deep learning model to predict wind generation based on historical wind generation and historical climate data. Then, the model is used to predict future wind generation under the RCP 2.6 and 8.5 scenarios using projected climate data for the respective scenarios.

The image below shows the detailed methodology.

![Methodology](Methodology.png)

## **Data Sources**
1. **Elexon** - Wind generation data per wind farm, from 2019 - 2024
2. **Climate Data Store, ERA5** - Historical climate data, 2019 - 2024
3. **Climate Data Store, CMIP5, CMIP6** - Project climate data under RCP 2.6 and RCP 8.5 scenarios, 2025 - 2045

<span style="color:red">To do list: - add links the above datasets? ; - ???</span>

<span style="color:red">General Note: we need to have a uniform heading + subheading structure</span>

## **Data Extraction**


<span style="color:red">*modify code below to import all packages at the start and define a Path which links to the github repo that all the csvs then use? - might be a waste of time tbh...*</span>

### **Offshore Windfarm Generation Data Extraction**

In [ ]:
#import packages:

import numpy as np
import pandas as pd
import requests
import csv
from io import StringIO
from datetime import datetime, timedelta
import os

##### **Extracting list of offshore windfarms in the UK from REPD dataset:**

This section reads in UK REPD csv data for Q3 Oct 2024* & extracts only the offshore wind farms - data was downloaded from: 

https://assets.publishing.service.gov.uk/media/673b215249ce28002166a93e/repd-q3-oct-2024.csv/preview

The names of these offshore windfarms will be used to cross-reference with data from the Power Station Dictionary & assign each windfarm the BMU IDs associated with it.

Note: some windfarms has several BMU units associated with them.

**Q4 2024 data was released since, however, no new offfshore widnfarms became operational in that time, therefore we continued using Q3 2024 given time constraints for data-processing (see analysis at the end of the **Offshore Windfarm Generation Data Extraction** section)*


In [ ]:
# Read in the downloaded csv from gov.uk website:
REDP_data = pd.read_csv("/Users/axelsyed/Desktop/ESDA/BENV0148 Advanced ML for Energy Systems/Project/repd-q3-oct-2024.csv", encoding='cp1252') #read in csv - this csv has been added to the github repository if needed for reference.

# Filter csv data for offshore wind only:
REDP_data_offshore = REDP_data[REDP_data["Technology Type"]=="Wind Offshore"]

# Create list of offshore windfarm names to visualise what has been collected:
offshore_names = REDP_data_offshore["Site Name"].to_list()
print("UK Offshore Windfarms collected:\n\n",offshore_names)

# Keep columns of interest:
columns_of_interest = ["Ref ID","Operator (or Applicant)","Site Name","Technology Type","Installed Capacity (MWelec)","Development Status (short)","County","Region","Country","X-coordinate","Y-coordinate","Planning Permission Granted","Under Construction","Operational"] #select columns we want to keep - selected via manual inspection of data.
filtered_REDP_data_offshore = REDP_data_offshore[columns_of_interest] #apply the column of interest filter to the df

# Save final csv - update for user's respective path:
"""
filtered_REDP_data_offshore.to_csv("/Users/axelsyed/Desktop/ESDA/BENV0148 Advanced ML for Energy Systems/Project/REPD Offshore Wind Farms.csv") #csv has been added to github repo, code further down reads it in directly!
"""

"REPD Offshore Wind Farms.csv" can be found in Github Repo

##### **Power Station Dictionary data processing:**

The Power System Dictionary can be found at: https://osuked.github.io/Power-Station-Dictionary/index.html

It is described as: 

*A pilot dictionary focused on improving the discoverability, linkage, and automated extraction of data relating to power stations on the GB system*

The dictionary contains:

*Power Plant IDs*

*Dataset listing the various IDs used to refer to the same power plants. These IDs can represent same-as relationships as well as sets which relate to different elements of the same plant e.g. a plant may comprise multiple generation units (referred to as BMUs in the UK).*

*Contributors	Ayrton Bourn (Author)*
*Licenses	CC-BY-4.0*

The REPD offshore windfarm data we extracted each have an REPD ID, which is also a column in the Power System Dictionary data (see screenshot of website below). We can use this REPD ID to cross-reference the offshore windfarm name to the Settlement BMU IDs associated with it. We can then use these BMU IDs to extract Elexon historical generation data using the Elexon API - it must be noted that offshore windfarms often have several BMU units associated with them, for balancing purposes. It must also be noted that some offshore windfarms have several REPD IDs (the REPD (new) column in Power Station Dictionary) associated with them, such as for East Anglia: https://osuked.github.io/Power-Station-Dictionary/objects/10300

![Alt text](./PowerDictImage.png)

The data table at the bottom of the screenshot above is continued below (scrolled to the right), to show all of its columns names:

![Alt text](./PowerDictPic2.png)

In [ ]:
# Read in html info from Power Station Dictionary dataset which includes REPD ID & BMU ID for UK Power Plants:

power_station_html = pd.read_html("https://osuked.github.io/Power-Station-Dictionary/dictionary.html") #html found by inspecting website.
print("From the html we obtain a:",type(power_station_html))

From the html we obtain a: <class 'list'>


By iterating through the list given by the html, the data table of interest is found:

In [ ]:
# The html link gives us a list - the third object in the list is the table with the info we want:
linked_ids = power_station_html[2]
print("The third object in the list is our data table of interest:\n\n",linked_ids) #you can compare output with screenshot above.

The third object in the list is our data table of interest:

      Dictionary ID                 GPPD ID ESAIL ID           Common Name  \
0            10000                       -     MARK  Rothes Bio-Plant CHP   
1            10001  GBR1000377, GBR1000369     DIDC                Didcot   
2            10002  GBR1000374, GBR1000375     ABTH            Aberthaw B   
3            10003              GBR1000142    COTPS                Cottam   
4            10004  GBR0000174, GBR1000112    DRAXX                  Drax   
..             ...                     ...      ...                   ...   
272          10313                       -        -                Calder   
273          10314                       -        -              Dounreay   
274          10315                       -        -           Trawsfynydd   
275          10316                       -        -             Windscale   
276          10317                       -        -              Winfrith   

             

Extract data from table which have an REPD ID entry and keep columns of interest - Common Name, Settlement BMU ID & REPD ID (new):

In [ ]:
# Filter rows where REPD ID (New) is not "-":
filtered_ids = linked_ids[linked_ids["REPD ID (New)"] != "-"]

# Select the columns of interested:
final_BMUs = filtered_ids[["Common Name", "Settlement BMU ID", "REPD ID (New)"]]

# Double-check:
print(final_BMUs.head())

                          Common Name     Settlement BMU ID REPD ID (New)
122  Aikengall 2 Wind Farm Generation  T_AKGLW-2, T_AKGLW-3          7015
123                   Airies Windfarm             E_AIRSW-1          4462
124               Andershaw Wind Farm             E_ASHWW-1          4110
125                An Suidhe Windfarm             T_ANSUW-1          3298
126                Arecleoch Windfarm             T_ARCHW-1          3258


In [ ]:
# Summarising the extracted data:

print("Number of windfarms extracted:",len(final_BMUs))

# This will count all BMU IDs across all cells, including those with multiple IDs
total_bmu_ids_1 = 0
for bmu_string in final_BMUs['Settlement BMU ID']:
    # Split by comma and count how many IDs are in this cell
    ids = [id.strip() for id in str(bmu_string).split(',') if id.strip()]
    total_bmu_ids_1 += len(ids)

print(f"Total number of Settlement BMU IDs (including multiple per cell): {total_bmu_ids_1}")

Number of windfarms extracted: 60
Total number of Settlement BMU IDs (including multiple per cell): 117


##### **Joining BMU IDs to the REPD Offshore Windfarm data:**

The dataframe of the REPD Offshore Wind Farms.csv that is read in, is expanded so that there is one REPD ID per row, the final_BMUs dataframe is then inner joined on REPD ID (New) [final_BMUs dataframe] = Ref ID [repd_offshore_df dataframe], and finally the merged dataframe is expanded to ensure there is one BMU unit per row:

In [ ]:
# Read the REPD Offshore Wind Farms CSV
repd_offshore_df = pd.read_csv("/Users/axelsyed/Desktop/ESDA/BENV0148 Advanced ML for Energy Systems/Project/REPD Offshore Wind Farms.csv")

# First expand by REPD ID (New)
repd_expanded_rows = []

for _, row in final_BMUs.iterrows():
    # Split the REPD ID values if there are multiple
    repd_ids = str(row["REPD ID (New)"]).split(',')
    
    # Create a new row for each ID
    for repd_id in repd_ids:
        # Clean the ID and try to convert to integer
        try:
            clean_id = int(repd_id.strip())
            new_row = row.copy()
            new_row["REPD ID (New)"] = clean_id
            repd_expanded_rows.append(new_row)
        except ValueError:
            # Skip any values that can't be converted to integers
            continue

# Create expanded DataFrame for REPD IDs
if repd_expanded_rows:
    repd_expanded_df = pd.DataFrame(repd_expanded_rows)
    
    # Join with the REPD Offshore Wind Farms data
    merged_df = pd.merge(
        repd_expanded_df,
        repd_offshore_df,
        left_on="REPD ID (New)",
        right_on="Ref ID",
        how="inner" #inner merge!
    )
    
    # Now expand by Settlement BMU ID
    bmu_expanded_rows = []
    
    for _, row in merged_df.iterrows():
        # Split the Settlement BMU ID values if there are multiple
        bmu_ids = str(row["Settlement BMU ID"]).split(',')
        
        # Create a new row for each BMU ID
        for bmu_id in bmu_ids:
            clean_bmu_id = bmu_id.strip()
            if clean_bmu_id:  # Skip empty strings
                new_row = row.copy()
                new_row["Settlement BMU ID"] = clean_bmu_id
                bmu_expanded_rows.append(new_row)
    
    # Create final expanded DataFrame with one BMU ID per row
    if bmu_expanded_rows:
        final_df = pd.DataFrame(bmu_expanded_rows)
        
        # Display the final result
        print("Final merged dataframe with one Settlement BMU ID per row:\n")
        print(final_df)
        print(f"Total number of rows in final dataframe: {len(final_df)}")
        print(f"Number of unique Settlement BMU IDs: {len(final_df['Settlement BMU ID'].unique())}")
    else:
        print("No valid Settlement BMU IDs found for expansion")
else:
    print("No valid REPD IDs found for joining")

# Keep the original REPD offshore dataframe intact
print("\nOriginal REPD Offshore Wind Farms dataframe (preserved):")
print(repd_offshore_df.head())
print(f"Total rows in original REPD dataframe: {len(repd_offshore_df)}")

#saving final merged csv as "merged_REPD_BMU_raw_data.csv":
"""
final_df.to_csv("/Users/axelsyed/Desktop/ESDA/BENV0148 Advanced ML for Energy Systems/Project/merged_REPD_BMU_raw_data.csv") #update path to user's path 
"""

Final merged dataframe with one Settlement BMU ID per row:

                             Common Name Settlement BMU ID  REPD ID (New)  \
0           Burbo Bank Offshore Windfarm           E_BURBO           2539   
0           Burbo Bank Offshore Windfarm         T_BRBEO-1           2539   
1           Burbo Bank Offshore Windfarm           E_BURBO           2487   
1           Burbo Bank Offshore Windfarm         T_BRBEO-1           2487   
2   Dudgeon Offshore Wind Farm Generator         T_DDGNO-1           2538   
..                                   ...               ...            ...   
30                               Hornsea         T_HOWAO-1           2525   
30                               Hornsea         T_HOWAO-2           2525   
30                               Hornsea         T_HOWAO-3           2525   
31                          Triton Knoll         T_TKNEW-1           2532   
31                          Triton Knoll         T_TKNWW-1           2532   

    Unnamed: 0 

It must be noted that the REPD csv data has multiple entries for the same windfarm, extra entries for a windfarm correspond to extensions of the windfarm (see "Site Name" column) - the only difference in the data are the dates in the "Installed capacity", "Planning Permission Granted", "Under Construction" & "Operational" columns.

Therefore, in the "merged_REPD_BMU_raw_data.csv" that is saved by the code above, duplicates had to be manually removed. The rows where the "Operational" date were most recent, were kept. Additional manual cleaning of the csv had to be done, for example, for the East Anglia windfarm, one of the joins led to Hornsea 1 being the site name (see screenshot below).

![Alt text](./EastAngliaPic1.png)
*Screenshot of a section of: "merged_REPD_BMU_raw_data.csv" - the inner joined CSV of REPD offshore windfarm data + BMU IDs from Power Station Dictionary data. The rows in yellow were ultimately deleted to remove duplicates. Red highlights a data issue*

*Note that the same BMU IDs were assigned to the same windfarm, which means our csv data processing works correctly*


In the screenshot above, we can see there is an issue with the join on  REPD ID (New)/Ref ID = 2525, which is assigns the East Anglia windfarm to: Orsted (formerly Dong Energy); for the Site Name: Hornsea 1 - Heron & Njord, an obvious error.

This issue arises from the fact Power Station Dictionary entry for East Anglia 1 does not have the correct REPD (new) assigned to it (see screenshot below).

![Alt text](./PowerDict_EA1.png)

This is confirmed by checking the REPD Q4 2024 entry for East Anglia 1:

![Alt text](./REPD_EA1.png)

Looking at the screenshot above, we can see the Ref ID for East Anglia 1 is **2524**, which doesn't appear in the Power Station Dictionary screenshot above! This issue was submitted to the developer & the correct data was manually entered in our merged csv.

The final, manually cleaned, "merged_REPD_BMU_raw_data.csv" was saved as: "Final BMU Clean Version.csv" (can be found in Github repo). The process ensured all of the BMU IDs appeared only once in the merged data.

##### **Section of code that extracts data for each BMU ID:**

URL for Elexon dataset we used:

https://bmrs.elexon.co.uk/api-documentation/endpoint/datasets/B1610/stream

Description:

*BMRS Datasets*

*Actual Generation Output Per Generation Unit (B1610) stream*
*This endpoint provides the actual metered volume output (MWh) per Settlement Period for all BM units (Positive, Negative or zero MWh values).*

*The information is published five days after the end of the operational period based on the Interim Information (II) Settlement Run and the data is refreshed by subsequent Settlement Runs.*

*The generation output returned by this endpoint is the metered volume and not the instantaneous power output as often appears in other specifications.*

The code below iterates through the "Final BMU Clean Version.csv" (can be found in Github repo) & for each "Settlement BMU ID", it queries the Elexon API and extracts the Actual Generation Output (MWh) for each half-hourly Settlement Period from the start of 2019 till the end of 2024 & saves the data - each file name has the associated windfarm and BMU in the title. If a windfarm's "Operational" date was after 01/01/2019, the generation data started collecting from the "Operational" date till the end of 2024.

In [ ]:
#Code to extract data using requests from Elexon API

# Output directory:
output_dir = "/Users/axelsyed/Desktop/ESDA/BENV0148 Advanced ML for Energy Systems/Project/BMU data/" #update to user's directory

# Load windfarm BMUs from the "Final BMU Clean Version.csv":
print("Loading BMU data.")
bmu_data = pd.read_csv("/Users/axelsyed/Desktop/ESDA/BENV0148 Advanced ML for Energy Systems/Project/Final BMU Clean Version.csv") #csv can be found in github repo

# Clean and prepare data
print(f"Found {bmu_data.shape[0]} records in BMU file")

# Get unique BMU IDs, excluding NaN or empty values
bmu_ids = bmu_data[bmu_data["Settlement BMU ID"].notna() & (bmu_data["Settlement BMU ID"] != "")]["Settlement BMU ID"].unique()
print(f"Found {len(bmu_ids)} unique BMU IDs")

# Function to convert DD/MM/YYYY to YYYY-MM-DD format
def convert_date_format(date_str):
    if pd.isna(date_str) or date_str == "":
        # If no operational date, use a default start date of 2018-01-01
        return "2018-01-01"
    try:
        date_obj = datetime.strptime(date_str, "%d/%m/%Y")
        return date_obj.strftime("%Y-%m-%d")
    except ValueError:
        print(f"Warning: Could not parse date '{date_str}', using default date")
        return "2018-01-01"

# Function to check if date is before "2019-01-01":

def check_date(start_date):
    if start_date < "2019-01-01":
        start_date = "2019-01-01"
        print("Plant operational since before 1st Jan 2019")
        return start_date
    else:
        print("Plant operational since:", start_date)
        return start_date

# Set end date for all queries
date_end = "2024-12-31T23%3A30Z" #ends at the end of 2024.

# Process each BMU ID
for i, bmu_id in enumerate(bmu_ids):
    print(f"Processing BMU {i+1}/{len(bmu_ids)}: {bmu_id}")
    
    # Get the BMU information
    bmu_info = bmu_data[bmu_data["Settlement BMU ID"] == bmu_id].iloc[0]
    common_name = bmu_info["Common Name"]
    
    # Get operational date and convert to API format
    operational_date = bmu_info["Operational"]
    date_start_intermediary = convert_date_format(operational_date)
    date_start = check_date(date_start_intermediary)
    # Construct API URL
    url = f"https://data.elexon.co.uk/bmrs/api/v1/datasets/B1610/stream?from={date_start}&to={date_end}&bmUnit={bmu_id}"
    
    # Sanitize filename (remove invalid characters)
    safe_name = "".join(c if c.isalnum() or c in "._- " else "_" for c in common_name)
    filename = f"{output_dir}/{safe_name}_{bmu_id}.csv"
    
    print(f"  Fetching data from {date_start} to 2023-12-31 for {bmu_id}")
    
    try:
        # Fetch data from API
        response = requests.get(url)
        
        if response.status_code == 200:
            # Parse JSON response
            df_bmu_data = pd.read_json(StringIO(response.text))
            
            # Check if data was returned
            if not df_bmu_data.empty:
                # Save to CSV
                df_bmu_data.to_csv(filename, index=False)
                print(f"  Successfully saved {df_bmu_data.shape[0]} records to {filename}")
            else:
                print(f"  Warning: No data returned for {bmu_id}")
                # Create empty file with headers to indicate processing was attempted
                pd.DataFrame(columns=["settlementDate", "settlementPeriod", "quantity", "bmUnit"]).to_csv(filename, index=False)
        else:
            print(f"  Error: API returned status code {response.status_code}")
            print(f"  Response: {response.text[:200]}...")  # prints first 200 characters of response
    except Exception as e:
        print(f"  Exception occurred: {str(e)}")

print("Processing complete!")

Loading BMU data.
Found 66 records in BMU file
Found 66 unique BMU IDs
Processing BMU 1/66: E_BURBO
Plant operational since before 1st Jan 2019
  Fetching data from 2019-01-01 to 2023-12-31 for E_BURBO
  Successfully saved 103680 records to /Users/axelsyed/Desktop/ESDA/BENV0148 Advanced ML for Energy Systems/Project/BMU data//Burbo Bank Offshore Wind Farm_E_BURBO.csv
Processing BMU 2/66: T_BRBEO-1
Plant operational since before 1st Jan 2019
  Fetching data from 2019-01-01 to 2023-12-31 for T_BRBEO-1
  Successfully saved 103680 records to /Users/axelsyed/Desktop/ESDA/BENV0148 Advanced ML for Energy Systems/Project/BMU data//Burbo Bank Offshore Wind Farm_T_BRBEO-1.csv
Processing BMU 3/66: T_DDGNO-1
Plant operational since before 1st Jan 2019
  Fetching data from 2019-01-01 to 2023-12-31 for T_DDGNO-1
  Successfully saved 103680 records to /Users/axelsyed/Desktop/ESDA/BENV0148 Advanced ML for Energy Systems/Project/BMU data//Dudgeon Offshore Wind Farm_T_DDGNO-1.csv
Processing BMU 4/66: T_

It must be noted that only 53 out of 66 Settlement BMU IDs returned data when the Elexon API was queried (see next section). However, this was not an problem as all offshore windfarms had at least one their BMU IDs which had some Elexon generation data associated with it. ⟹ This is analysed in the "Analysis of BMU not collecting.csv" (can be found in Github repo) which was created from the "Final BMU Clean Version.csv" with an extra "Data Retrieved?" column added (YES/NO answers manually entered by looking at output of the cell above).

The generation datasets for each BMU extracted above can be found in the Github repo under: BENV0148_Group_3/Data/wind_gen_BMU_data

### **Historical Climate Data Extraction**

The historical climate data is extracted from the "Climate Data Store" under the ERA5 model.

Data is extracted for 2019-2024 as that is the period for which historical wind generation data is extracted.

Note that certain wind farms act as multiple Balancing Mechanism Units (BMUs).

In [17]:
# Import the required packages 
# If needed, first install the packages using "!pip install package_name" (replace package_name with the name of the package)
import cdsapi
import numpy as np
import pandas as pd
import requests
import csv
from io import StringIO
from datetime import datetime, timedelta
import os

In [18]:
# Import the csv file containing all wind farm BMUs & locations
bmu_data_analysed = pd.read_csv("https://raw.githubusercontent.com/OkeMoyo/BENV0148_Group_3/main/Analysis%20of%20BMU%20not%20collecting.csv",
encoding="ISO-8859-1")

In [ ]:
# Check the number of wind farms in the data
print("There are ", len(bmu_data_analysed), "total BMUs in the dataset.")

# Inspect the data
bmu_data_analysed.head()

There are  66 total BMUs in the dataset.


,Unnamed: 0.1,Common Name,Settlement BMU ID,Data Retrieved?,REPD ID (New),Unnamed: 0,Ref ID,Operator (or Applicant),Site Name,Technology Type,...,Development Status (short),County,Region,Country,X-coordinate,Y-coordinate,Planning Permission Granted,Under Construction,Operational,Data Retrieved?.1
0,0,Burbo Bank Offshore Wind Farm,E_BURBO,YES,2539,2061,2539,Orsted (formerly Dong Energy),Burbo Bank Extension (Burbo Bank 2),Wind Offshore,...,Operational,Offshore,Offshore,England,315815,398892,10/12/2014,10/06/2016,27/04/2017,NaN
1,0,Burbo Bank Offshore Wind Farm,T_BRBEO-1,YES,2539,2061,2539,Orsted (formerly Dong Energy),Burbo Bank Extension (Burbo Bank 2),Wind Offshore,...,Operational,Offshore,Offshore,England,315815,398892,10/12/2014,10/06/2016,27/04/2017,NaN
2,2,Dudgeon Offshore Wind Farm,T_DDGNO-1,YES,2538,2060,2538,Statoil / Statkraft,Dudgeon East,Wind Offshore,...,Operational,Offshore,Offshore,England,575000,361000,06/07/2012,17/03/2016,15/10/2017,NaN
3,2,Dudgeon Offshore Wind Farm,T_DDGNO-2,YES,2538,2060,2538,Statoil / Statkraft,Dudgeon East,Wind Offshore,...,Operational,Offshore,Offshore,England,575000,361000,06/07/2012,17/03/2016,15/10/2017,NaN
4,2,Dudgeon Offshore Wind Farm,T_DDGNO-3,YES,2538,2060,2538,Statoil / Statkraft,Dudgeon East,Wind Offshore,...,Operational,Offshore,Offshore,England,575000,361000,06/07/2012,17/03/2016,15/10/2017,NaN


In [20]:
# Filter out the wind farms whose data were successfully extracted
bmu_data_filtered = bmu_data_analysed[bmu_data_analysed["Data Retrieved?"] == "YES"]
print(len(bmu_data_filtered), "of", len(bmu_data_analysed),"BMUs' data were successfully extracted from Elexon.")
bmu_data_filtered.head()

53 of 66 BMUs' data were successfully extracted from Elexon.


,Unnamed: 0.1,Common Name,Settlement BMU ID,Data Retrieved?,REPD ID (New),Unnamed: 0,Ref ID,Operator (or Applicant),Site Name,Technology Type,...,Development Status (short),County,Region,Country,X-coordinate,Y-coordinate,Planning Permission Granted,Under Construction,Operational,Data Retrieved?.1
0,0,Burbo Bank Offshore Wind Farm,E_BURBO,YES,2539,2061,2539,Orsted (formerly Dong Energy),Burbo Bank Extension (Burbo Bank 2),Wind Offshore,...,Operational,Offshore,Offshore,England,315815,398892,10/12/2014,10/06/2016,27/04/2017,NaN
1,0,Burbo Bank Offshore Wind Farm,T_BRBEO-1,YES,2539,2061,2539,Orsted (formerly Dong Energy),Burbo Bank Extension (Burbo Bank 2),Wind Offshore,...,Operational,Offshore,Offshore,England,315815,398892,10/12/2014,10/06/2016,27/04/2017,NaN
2,2,Dudgeon Offshore Wind Farm,T_DDGNO-1,YES,2538,2060,2538,Statoil / Statkraft,Dudgeon East,Wind Offshore,...,Operational,Offshore,Offshore,England,575000,361000,06/07/2012,17/03/2016,15/10/2017,NaN
3,2,Dudgeon Offshore Wind Farm,T_DDGNO-2,YES,2538,2060,2538,Statoil / Statkraft,Dudgeon East,Wind Offshore,...,Operational,Offshore,Offshore,England,575000,361000,06/07/2012,17/03/2016,15/10/2017,NaN
4,2,Dudgeon Offshore Wind Farm,T_DDGNO-3,YES,2538,2060,2538,Statoil / Statkraft,Dudgeon East,Wind Offshore,...,Operational,Offshore,Offshore,England,575000,361000,06/07/2012,17/03/2016,15/10/2017,NaN


In [21]:
# The datasets contain 53 BMUs, but how many of them are unique wind farms?
unique_names = bmu_data_filtered["Common Name"].unique()
print("There are", len(unique_names), "unique wind farms in the dataset")

There are 21 unique wind farms in the dataset


In [22]:
# Inspect the array of unique wind farms
unique_names

array(['Burbo Bank Offshore Wind Farm', 'Dudgeon Offshore Wind Farm',
       'Galloper Offshore Wind Farm',
       'Greater Gabbard Offshore Wind Farm',
       'Gwynt y Mor Offshore Wind Farm', 'Humber Offshore Wind Farm',
       'Lincs Offshore Wind Farm', 'London Array Wind Farm',
       'Ormonde Offshore Wind Farm', 'Race Bank Offshore Wind Farm',
       'Rampion Offshore Wind Farm', 'Sheringham Shoals Wind Farm',
       'Thanet Offshore Wind Farm', 'Walney Offshore Wind Farm',
       'Westermost Rough Wind Farm',
       'West of Duddon Sands Offshore Wind Farm',
       'Aberdeen Offshore Wind Farm', 'Beatrice Offshore Wind Farm',
       'East Anglia Offshore Wind Farm', 'Hornsea Offshore Wind Farm',
       'Triton Knoll Offshore Wind Farm'], dtype=object)

There are 21 unique offshore wind farms in the dataset for which data was able to be extracted from Elexon. This volume of data is acceptable for this project as not all wind farms participate in the UK electricity balancing market.

The x and y coordintaes of the wind farm locations in csv file are in the British National Grid (OSGB36) coordinate system, representing eastings (X) and northings (Y), respectively.

In [ ]:
# Extract the x and y coordinates for each unique wind farm in the filtered dataset

for name in unique_names:
    # Get the first occurrence of this Common Name
    row = bmu_data_filtered[bmu_data_filtered["Common Name"] == name].iloc[0]
    
    # Extract coordinates
    x_coord = row['X-coordinate']
    y_coord = row['Y-coordinate']
    
    # Print the result
    print(f"{name} is located at: X = {x_coord} eastings, Y = {y_coord} northings")

To extract the historical climate data for each unique wind farm, the coordinates are transformed from the BNG system to the WGS84 system for compatibility with the requirements of the climate data store. A function is created to generate bounding boxes specifying the location of the wind farms in the WGS84 coordinate system to enable data extraction from the climate data store.

In [12]:
# bounding box function that creates bounding box depending on windfarm location:
from pyproj import Transformer
# Create transformer: BNG (EPSG:27700) → WGS84 (EPSG:4326)
transformer = Transformer.from_crs("epsg:27700", "epsg:4326", always_xy=True)

def create_bounding_box(x_coord, y_coord,resolution):
   
    # Transform to (lon, lat)
    lon, lat = transformer.transform(x_coord, y_coord)

    # Create box centered on the point with size of one CMIP grid cell
    # Return box as [North, West, South, East]
    return [
        lat + (resolution/2),  # North
        lon - (resolution/2),  # West
        lat - (resolution/2),  # South
        lon + (resolution/2)   # East
    ]

References for spatial resolution used above:

https://confluence.ecmwf.int/display/CKB/ERA5%3A+data+documentation#heading-Spatialgrid

https://confluence.ecmwf.int/display/CKB/ERA5%3A+What+is+the+spatial+reference


<span style="color:red">*include spatial resolution discussion here & how it isn;t a limitation?*</span>

In [ ]:
# Extract the historical climate data (2019-2024) for each wind farm using the climate data store api and the bounding box function
# Change all file paths below to your preferred directory

from math import cos, radians

# Create a directory to store the data
if not os.path.exists('era5_data'):
    os.makedirs('era5_data')

# Create a directory to store the data
output_dir = "C:/Users/Oke/Documents/UCL MSc ESDA/ESDA_Term 2/BENV0148 Advanced Machine Learning/BENV0148 Coursework/ERA5 Historical Data"

# Dataset name
dataset = "reanalysis-era5-single-levels"

#Create client object
client = cdsapi.Client()

# Base request template
base_request = {
    "product_type": ["reanalysis"],
    "variable": [
        "10m_u_component_of_wind",
        "10m_v_component_of_wind",
        "2m_dewpoint_temperature",
        "2m_temperature",
        "surface_pressure"
    ],
    "year": ["2019", "2020", "2021", "2022", "2023", "2024"],
    "month": [
        "01", "02", "03", "04", "05", "06",
        "07", "08", "09", "10", "11", "12"
    ],
    "day": [
        "01", "02", "03", "04", "05", "06",
        "07", "08", "09", "10", "11", "12",
        "13", "14", "15", "16", "17", "18",
        "19", "20", "21", "22", "23", "24",
        "25", "26", "27", "28", "29", "30", "31"
    ],
    "time": [
        "00:00", "01:00", "02:00", "03:00", "04:00", "05:00",
        "06:00", "07:00", "08:00", "09:00", "10:00", "11:00",
        "12:00", "13:00", "14:00", "15:00", "16:00", "17:00",
        "18:00", "19:00", "20:00", "21:00", "22:00", "23:00"
    ],
    "format": "grib" 
}

# Process each unique wind farm
for farm_name in unique_names:
    # Get the first row with this farm name
    row = bmu_data_filtered[bmu_data_filtered['Common Name'] == farm_name].iloc[0]
    
    # Extract coordinates
    x_coord = row['X-coordinate']
    y_coord = row['Y-coordinate']
    
    # Skip if coordinates are missing
    if pd.isna(x_coord) or pd.isna(y_coord):
        print(f"Skipping {farm_name} due to missing coordinates")
        continue
    
    print(f"Processing {farm_name} at coordinates ({x_coord}, {y_coord})")
    
    # Create a bounding box at ERA5 native resolution
    area_box = create_bounding_box(x_coord, y_coord, 0.28125) # ERA5 resolution is ~0.28125 degrees
    print(f"  ERA5 bounding box: {area_box}")
    
    # Create a custom request for this farm
    farm_request = base_request.copy()
    farm_request["area"] = area_box
    
    # Set output filename - clean up farm name for file safety
    safe_farm_name = ''.join(c if c.isalnum() else '_' for c in farm_name)
    target = os.path.join(output_dir, f"{safe_farm_name}.grib")
    
    try:
        print(f"Downloading ERA5 data for {farm_name}...")
        client.retrieve(dataset, farm_request).download(target)
        print(f"Successfully downloaded data to {target}")
    except Exception as e:
        print(f"Error downloading data for {farm_name}: {str(e)}")

print("Processing complete!")


### **Projected Climate Data Extraction**

##### 1. Projected climate data from the CMIP6 dataset on CDS is only available in monthly resolution.
##### 2. For projecting future wind power generation, upsampling techniques may be used to convert the monthly projected climate data to daily projected climate data to match the resolution of daily historic climate data used. 
##### 3. For consistency, the same function for extraction is used for the historic climate data, with the only difference being adjustments to the base_request template function namely to extract from CMIP6 dataset (projected climate data) instead of ERA5 (historic climate data) and for the years 2025-2045 at a monthly resolution. 
##### 4. Since not all climate projection models cover the all the regions where offshore GB wind farms are located, a loop is used for each windfarm to scan all projection models and find the suitable corresponding model that has data available for the geographic location of that particular windfarm.
##### 5. The same cell below is used to extract data for both scenarios (ssp1-2.6 and ssp5-8.5). To switch between extracting data for the two different scenarios the "experiment" variable in the first line of the cell is uncommented as needed. 

In [ ]:
#UNCOMMENT EITHER OF THE LINES BELOW TO SWITCH BETWWEEN SSP126 AND SSP585 DATA

experiment = 'ssp126'
#experiment = 'ssp585'

# Create a directory to store the data
output_dir = r"C:\Users\LGA\Desktop\UCL-ESDA coursework\TERM 2\BENV0148 - Advanced Machine Learning for Energy Systems\Coursework\Data\gitclone\BENV0148_Group_3\Data\projected_climate_data\ssp126"
dataset = 'projections-cmip6'

base_request1 = {
                        'format': 'netcdf',
                        'temporal_resolution': 'monthly',
                        'experiment': experiment,
                        'model': 'HadGEM3-GC31-LL',
                        'ensemble_member': 'r1i1p1f1',
                         'variable': [
                                    'near_surface_air_temperature',
                                    'eastward_near_surface_wind',
                                    'northward_near_surface_wind',
                                    'surface_air_pressure',
                                    'specific_humidity'
                                    ],
                        'year': [str(y) for y in range(2025, 2046)],
                        'month': [f'{m:02d}' for m in range(1, 13)],
                    }
c = cdsapi.Client()


# Try a few models and ensemble members
models = ['HadGEM3-GC31-LL', 'CNRM-CM6-1', 'EC-Earth3', 'MIROC6']
ensemble_members = ['r1i1p1f1', 'r2i1p1f1', 'r3i1p1f1']


unique_names = bmu_data_filtered["Common Name"].unique()
for farm_name in unique_names:
    # Get the first row with this farm name
    row = bmu_data_filtered[bmu_data_filtered['Common Name'] == farm_name].iloc[0]
    
    # Extract coordinates
    x_coord = row['X-coordinate']
    y_coord = row['Y-coordinate']

    #Extract BMU-ID
    bmu_id = row['Settlement BMU ID']
    
    # Skip if coordinates are missing
    if pd.isna(x_coord) or pd.isna(y_coord):
        print(f"Skipping {farm_name} due to missing coordinates")
        continue
    
    print(f"Processing {farm_name} at coordinates ({x_coord}, {y_coord})")
    
    # Create a bounding box at CMIP native resolution
    area_box = create_bounding_box(x_coord, y_coord, 1.25) # ERA5 resolution is 1.25 degrees
    
    # Create a custom request for this farm
    farm_request = base_request1.copy()
    farm_request["area"] = area_box
    
    # Set output filename - clean up farm name for file safety
    safe_farm_name = ''.join(c if c.isalnum() else '_' for c in farm_name)
    target = os.path.join(output_dir, f"{safe_farm_name}.nc")

    #For the given wind farm, try all models and ensemble members until one works
    
    success = False # set a flag to track if a compatible model+ensemble combination is found
    # Loop through models and ensemble members
    for model in models:
        for ensemble in ensemble_members:
            farm_request["model"] = model
            farm_request["ensemble_member"] = ensemble

            try:
                print(f"Trying {model} | {ensemble} for {farm_name}")
                c.retrieve(dataset, farm_request).download(target),
                print(f"✅ Success: {model} | {ensemble}")
                success = True
            except Exception as e:
                print(f"❌ Failed: {model} | {ensemble}")
                print(f"Error downloading data for {farm_name}: {str(e)}")

            if success:
                break #if successful model+ensemble combination is found, move to next wind farm
        if success:  #if successful model+ensemble combination is found, move to next wind farm
            break


### **Data Pre-Processing**

#### **Historical Climate data**
The snippet of code below concatenates the historic climate data for each windfarm

In [ ]:
import xarray as xr
from collections import defaultdict

base_path = r"C:\Users\LGA\Desktop\UCL-ESDA coursework\TERM 2\BENV0148 - Advanced Machine Learning for Energy Systems\Coursework\Data\gitclone\BENV0148_Group_3\Data\historic_climate_data"

# Dictionary to store each windfarm's datasets
windfarm_data = defaultdict(list)

for year_folder in sorted(os.listdir(base_path)):
    year_path = os.path.join(base_path, year_folder)
    if os.path.isdir(year_path):
        for file in os.listdir(year_path):
            if file.endswith(".grib") and not file.endswith(".grib.5b7b6.idx"):

                file_path = os.path.join(year_path, file)
                print(f"📂 Reading file: {file_path}")  # 👈 This line logs the file being read
                try:
                    ds = xr.open_dataset(file_path, engine="cfgrib")
                    year = int(year_folder.split('_')[-1])  # Extract year
                    ds = ds.assign_coords(Year=year)
                    windfarm_data[file].append(ds)
                except Exception as e:
                    print(f"Could not read {file_path}: {e}")

# Concatenate each windfarm’s datasets
final_windfarm_datasets = {
    name.replace('.grib', '').replace('.grb', ''): xr.concat(datasets, dim="time")
    for name, datasets in windfarm_data.items()
}


#### Converting to CSVs
The cell below converts the grib files to CSVs for easy reading. No need to run, if the CSVs are already accessible through git

In [ ]:
import xarray as xr

base_path = r"C:\Users\LGA\Desktop\UCL-ESDA coursework\TERM 2\BENV0148 - Advanced Machine Learning for Energy Systems\Coursework\Data\gitclone\BENV0148_Group_3\Data\historic_climate_data" # CHANGE THIS TO YOUR FOLDER

for year_folder in sorted(os.listdir(base_path)):
    year_path = os.path.join(base_path, year_folder)

    if os.path.isdir(year_path):
        for file in os.listdir(year_path):
            if file.endswith(".grib") and not file.endswith(".grib.idx"):
                file_path = os.path.join(year_path, file)
                csv_path = file_path.replace(".grib", ".csv")

                if os.path.exists(csv_path):
                    print(f"🔁 Skipping (already converted): {csv_path}")
                    continue

                print(f"📂 Reading: {file_path}")
                try:
                    ds = xr.open_dataset(file_path, engine="cfgrib")
                    df = ds.to_dataframe().reset_index()
                    df.to_csv(csv_path, index=False)
                    print(f"✅ Saved CSV: {csv_path}")
                except Exception as e:
                    print(f"❌ Failed: {file_path}\n   Reason: {e}")


#### Concatenating the CSVs for each wind farm across the years

For each windfarm, we produce a single dataframe of historic climate data from 2019-2023

In [ ]:
from collections import defaultdict

base_path = r"C:\Users\LGA\Desktop\UCL-ESDA coursework\TERM 2\BENV0148 - Advanced Machine Learning for Energy Systems\Coursework\Data\gitclone\BENV0148_Group_3\Data\historic_climate_data" # CHANGE THIS TO YOUR FOLDER

# Dictionary to collect dataframes by windfarm
windfarm_csv_data = defaultdict(list)

for year_folder in sorted(os.listdir(base_path)):
    year_path = os.path.join(base_path, year_folder)

    if os.path.isdir(year_path):
        for file in os.listdir(year_path):
            if file.endswith(".csv"):
                file_path = os.path.join(year_path, file)
                windfarm_name = file.replace(".csv", "")

                try:
                    df = pd.read_csv(file_path)
                    df["Year"] = year_folder.split("_")[-1]  # Add year column
                    windfarm_csv_data[windfarm_name].append(df)
                    print(f"📥 Loaded: {file_path}")
                except Exception as e:
                    print(f"❌ Failed to read {file_path}: {e}")

# Now concatenate all years into one dataframe per windfarm
final_windfarm_dfs = {}

for windfarm_name, dfs in windfarm_csv_data.items():
    final_df = pd.concat(dfs, ignore_index=True)
    final_windfarm_dfs[windfarm_name] = final_df
    print(f"✅ Combined data for: {windfarm_name} ({len(final_df)} rows)")


In [ ]:
windfarm_names = list(final_windfarm_dfs.keys())
print(windfarm_names)


['Aberdeen_Offshore_Wind_Farm', 'Beatrice_Offshore_Wind_Farm', 'Burbo_Bank_Offshore_Wind_Farm', 'Dudgeon_Offshore_Wind_Farm', 'East_Anglia_Offshore_Wind_Farm', 'Galloper_Offshore_Wind_Farm', 'Greater_Gabbard_Offshore_Wind_Farm', 'Gwynt_y_Mor_Offshore_Wind_Farm', 'Hornsea_Offshore_Wind_Farm', 'Humber_Offshore_Wind_Farm', 'Lincs_Offshore_Wind_Farm', 'London_Array_Wind_Farm', 'Ormonde_Offshore_Wind_Farm', 'Race_Bank_Offshore_Wind_Farm', 'Rampion_Offshore_Wind_Farm', 'Sheringham_Shoals_Wind_Farm', 'Thanet_Offshore_Wind_Farm', 'Triton_Knoll_Offshore_Wind_Farm', 'Walney_Offshore_Wind_Farm', 'Westermost_Rough_Wind_Farm', 'West_of_Duddon_Sands_Offshore_Wind_Farm']


##### Access example - Aberdeen

In [ ]:


# Access data for one windfarm
aberdeen_df = final_windfarm_dfs["Aberdeen_Offshore_Wind_Farm"]
aberdeen_df.head(10)


['Aberdeen_Offshore_Wind_Farm', 'Beatrice_Offshore_Wind_Farm', 'Burbo_Bank_Offshore_Wind_Farm', 'Dudgeon_Offshore_Wind_Farm', 'East_Anglia_Offshore_Wind_Farm', 'Galloper_Offshore_Wind_Farm', 'Greater_Gabbard_Offshore_Wind_Farm', 'Gwynt_y_Mor_Offshore_Wind_Farm', 'Hornsea_Offshore_Wind_Farm', 'Humber_Offshore_Wind_Farm', 'Lincs_Offshore_Wind_Farm', 'London_Array_Wind_Farm', 'Ormonde_Offshore_Wind_Farm', 'Race_Bank_Offshore_Wind_Farm', 'Rampion_Offshore_Wind_Farm', 'Sheringham_Shoals_Wind_Farm', 'Thanet_Offshore_Wind_Farm', 'Triton_Knoll_Offshore_Wind_Farm', 'Walney_Offshore_Wind_Farm', 'Westermost_Rough_Wind_Farm', 'West_of_Duddon_Sands_Offshore_Wind_Farm']


,time,latitude,longitude,number,step,surface,valid_time,u10,v10,d2m,t2m,sp,Year
0,2019-01-01 00:00:00,57.256,-2.214,0,0 days,0.0,2019-01-01 00:00:00,7.651512,-4.679920,275.18616,280.91525,101140.030,2019
1,2019-01-01 00:00:00,57.256,-1.963,0,0 days,0.0,2019-01-01 00:00:00,7.865990,-4.702084,275.77222,281.01923,101912.410,2019
2,2019-01-01 00:00:00,57.005,-2.214,0,0 days,0.0,2019-01-01 00:00:00,5.542198,-4.630261,275.31090,281.83804,101401.375,2019
3,2019-01-01 00:00:00,57.005,-1.963,0,0 days,0.0,2019-01-01 00:00:00,5.726341,-4.917637,275.94050,282.00885,102243.470,2019
4,2019-01-01 01:00:00,57.256,-2.214,0,0 days,0.0,2019-01-01 01:00:00,7.496879,-4.672980,275.10327,280.48312,101241.660,2019
5,2019-01-01 01:00:00,57.256,-1.963,0,0 days,0.0,2019-01-01 01:00:00,7.951285,-4.550590,275.58330,280.68140,102010.160,2019
6,2019-01-01 01:00:00,57.005,-2.214,0,0 days,0.0,2019-01-01 01:00:00,5.359855,-3.996467,274.82648,280.88406,101507.060,2019
7,2019-01-01 01:00:00,57.005,-1.963,0,0 days,0.0,2019-01-01 01:00:00,5.824698,-4.053016,275.30615,281.14014,102346.125,2019
8,2019-01-01 02:00:00,57.256,-2.214,0,0 days,0.0,2019-01-01 02:00:00,7.111711,-4.728118,275.13495,280.26086,101357.940,2019
9,2019-01-01 02:00:00,57.256,-1.963,0,0 days,0.0,2019-01-01 02:00:00,7.642045,-4.605590,275.66974,280.54553,102119.875,2019


#### **Grouping historic climate data by hour for each windfarm**

As seen above, for each timestamp of a given windfarm, four rows of climate data are shown for four sets of coordinates within the bounding box. We group by hour and spatially AVERAGE the climate variables.


In [ ]:
hourly_windfarm_dfs = {}

for windfarm_name, df in final_windfarm_dfs.items():
    print(f"⏱️ Aggregating hourly for: {windfarm_name}")

    # Ensure 'time' is in datetime format
    df['time'] = pd.to_datetime(df['time'])

    # Round down to the hour (e.g., 2019-01-01 01:43 → 2019-01-01 01:00)
    df['hour'] = df['time'].dt.floor('H')

    # Group by each hour and take the mean of numeric columns
    hourly_df = df.groupby('hour').mean(numeric_only=True).reset_index()

    # Store result
    hourly_windfarm_dfs[windfarm_name] = hourly_df

    print(f"✅ Aggregated {windfarm_name}: {len(hourly_df)} hourly records")


##### Example - accessing Aberdeen historical concatenated data

In [44]:
# List available windfarms
print(list(hourly_windfarm_dfs.keys()))

# Access data for one windfarm
aberdeen_df = hourly_windfarm_dfs["Aberdeen_Offshore_Wind_Farm"]
aberdeen_df.head()


['Aberdeen_Offshore_Wind_Farm', 'Beatrice_Offshore_Wind_Farm', 'Burbo_Bank_Offshore_Wind_Farm', 'Dudgeon_Offshore_Wind_Farm', 'East_Anglia_Offshore_Wind_Farm', 'Galloper_Offshore_Wind_Farm', 'Greater_Gabbard_Offshore_Wind_Farm', 'Gwynt_y_Mor_Offshore_Wind_Farm', 'Hornsea_Offshore_Wind_Farm', 'Humber_Offshore_Wind_Farm', 'Lincs_Offshore_Wind_Farm', 'London_Array_Wind_Farm', 'Ormonde_Offshore_Wind_Farm', 'Race_Bank_Offshore_Wind_Farm', 'Rampion_Offshore_Wind_Farm', 'Sheringham_Shoals_Wind_Farm', 'Thanet_Offshore_Wind_Farm', 'Triton_Knoll_Offshore_Wind_Farm', 'Walney_Offshore_Wind_Farm', 'Westermost_Rough_Wind_Farm', 'West_of_Duddon_Sands_Offshore_Wind_Farm']


,hour,latitude,longitude,number,surface,u10,v10,d2m,t2m,sp
0,2019-01-01 00:00:00,57.1305,-2.0885,0.0,0.0,6.696510,-4.732475,275.552445,281.445342,101674.32125
1,2019-01-01 01:00:00,57.1305,-2.0885,0.0,0.0,6.658179,-4.318263,275.204800,280.797180,101776.25125
2,2019-01-01 02:00:00,57.1305,-2.0885,0.0,0.0,6.554337,-4.067016,275.217807,280.521988,101890.31375
3,2019-01-01 03:00:00,57.1305,-2.0885,0.0,0.0,6.623036,-3.949683,275.290970,280.336948,101971.28875
4,2019-01-01 04:00:00,57.1305,-2.0885,0.0,0.0,6.699568,-4.152366,275.330078,280.151777,102042.31250


#### **Historic wind power generation Data processing**
1) The data is aggregated from half hourly to hourly. For example, a ceiling function adds the wind power generated for the first 2 settlement periods, so 00:30 + 01:00 → 01:00.
2) The wind power generated is added for all the BMUs of a given windfarm for each time stamp

In [ ]:
from collections import defaultdict
import re

# Path to your wind generation CSVs
gen_folder = r"C:\Users\LGA\Desktop\UCL-ESDA coursework\TERM 2\BENV0148 - Advanced Machine Learning for Energy Systems\Coursework\Data\gitclone\BENV0148_Group_3\Data\wind_gen_BMU_data"

# Step 1: Prepare a defaultdict to collect dataframes per windfarm
hourly_generation_dfs = defaultdict(list)

# Normalize filenames: strip BMU ID suffix after _T_ or _E_
def normalize_windfarm_name(filename):
    base = re.split(r'_T_|_E_', filename)[0]
    return base.strip().replace(" ", "_")

# Step 2: Read and process each CSV
for file in os.listdir(gen_folder):
    if file.endswith(".csv"):
        file_path = os.path.join(gen_folder, file)
        windfarm_key = normalize_windfarm_name(file)

        try:
            df = pd.read_csv(file_path)
            df["halfHourEndTime"] = pd.to_datetime(df["halfHourEndTime"])

            # Ceil timestamps so 00:30 + 01:00 → 01:00
            df["hour"] = df["halfHourEndTime"].dt.ceil("H")

            # Sum only the "quantity" column
            hourly_df = df.groupby("hour", as_index=False)["quantity"].sum()

            # Optionally keep other info like BMU_ID if needed (commented for now)
            # hourly_df["BMU_ID"] = df.groupby("hour")["BMU_ID"].first().values

            hourly_generation_dfs[windfarm_key].append(hourly_df)

            print(f"✅ Processed {file} under key '{windfarm_key}'")

        except Exception as e:
            print(f"❌ Failed to process {file}: {e}")

# Step 3: Combine BMUs for each windfarm
final_generation_dfs = {}

for windfarm_key, dfs in hourly_generation_dfs.items():
    combined_df = pd.concat(dfs).groupby("hour", as_index=False).sum()
    final_generation_dfs[windfarm_key] = combined_df
    print(f"📦 Final combined generation for: {windfarm_key}")


#### Example - access historic windfarm generation for Aberdeen

In [46]:
final_generation_dfs["Aberdeen_Offshore_Wind_Farm"].head(10)  # Example to check the data for one windfarm

,hour,quantity
0,2019-02-01 01:00:00,3.666
1,2019-02-01 02:00:00,4.424
2,2019-02-01 03:00:00,7.314
3,2019-02-01 04:00:00,20.394
4,2019-02-01 05:00:00,24.833
5,2019-02-01 06:00:00,6.744
6,2019-02-01 07:00:00,7.788
7,2019-02-01 08:00:00,5.234
8,2019-02-01 09:00:00,5.109
9,2019-02-01 10:00:00,2.318


### **Producing Final Merged Dataset**
So far we have:

1) hourly_windfarm_dfs: A data dictionary with historic hourly  climate data for all offshore windfarms for the years 2019-2023

2) final_generation_dfs: A data dictionary with historic hourly  wind generation data for all offshore windfarms for the years 2019-2023

We want to produce a final "merged_data" data dcitionary that combines the two above for each windfarm: combined dataset of climate + generation for modeling or analysis

In [ ]:
merged_data = {}

for windfarm_key, climate_df in hourly_windfarm_dfs.items():
    # Some windfarms may not have matching generation data
    if windfarm_key in final_generation_dfs:
        gen_df = final_generation_dfs[windfarm_key]
        
        merged_df = pd.merge(climate_df, gen_df, left_on="hour", right_on="hour", how="inner")
        merged_data[windfarm_key] = merged_df
        print(f"🔗 Merged: {windfarm_key} ({len(merged_df)} rows)")
    else:
        print(f"⚠️ No generation data for: {windfarm_key}")


#### Example - Accessing merged data for Aberdeen offshore windfarm

In [48]:
merged_data["Aberdeen_Offshore_Wind_Farm"].head(10)  # Example to check the data for one windfarm

,hour,latitude,longitude,number,surface,u10,v10,d2m,t2m,sp,quantity
0,2019-02-01 01:00:00,57.1305,-2.0885,0.0,0.0,3.347781,-1.712159,270.595155,272.952145,98817.36750,3.666
1,2019-02-01 02:00:00,57.1305,-2.0885,0.0,0.0,3.270191,-1.573564,270.255047,272.929200,98824.22625,4.424
2,2019-02-01 03:00:00,57.1305,-2.0885,0.0,0.0,3.253275,-1.333238,269.973372,272.894290,98839.03000,7.314
3,2019-02-01 04:00:00,57.1305,-2.0885,0.0,0.0,3.394168,-1.272390,270.094055,273.110963,98855.18750,20.394
4,2019-02-01 05:00:00,57.1305,-2.0885,0.0,0.0,3.441805,-1.288301,270.861803,273.754715,98840.80375,24.833
5,2019-02-01 06:00:00,57.1305,-2.0885,0.0,0.0,3.371131,-1.281039,271.457212,273.948363,98876.45875,6.744
6,2019-02-01 07:00:00,57.1305,-2.0885,0.0,0.0,3.147674,-1.248965,271.934835,274.146568,98878.21750,7.788
7,2019-02-01 08:00:00,57.1305,-2.0885,0.0,0.0,2.852993,-1.166969,271.980537,274.180465,98955.70875,5.234
8,2019-02-01 09:00:00,57.1305,-2.0885,0.0,0.0,2.699424,-1.001670,272.095620,274.414578,98999.29000,5.109
9,2019-02-01 10:00:00,57.1305,-2.0885,0.0,0.0,1.982153,-0.623916,271.787375,274.484920,99032.79750,2.318
